In [1]:
import folium
import pandas as pd
import geopandas as gpd
from shapely import wkt 

In [2]:
from turbodbc import connect
connection = connect(dsn='Teradata')

In [3]:
query = """SELECT OSM_id, CAST(MULTIPOLYGON.ST_ASTEXT() AS VARCHAR(3000)) polygon 
        FROM UAT_DM.OSM_RUSSIA_MULTIPOLYGONS
        WHERE BUILDING_TAG IS NOT NULL
        AND MULTIPOLYGON.ST_MBR_Xmin() < 0"""

In [4]:
cursor = connection.cursor()
cursor.execute(query)
df = pd.DataFrame(cursor.fetchallnumpy())

In [5]:
df.head()

,OSM_ID,polygon
0,281927594,"POLYGON ((-179.400757 68.8866838,-179.40099090..."
1,309129277,"POLYGON ((-175.8397009 65.0106686,-175.8395906..."
2,129101699,"POLYGON ((-173.212450499999989 64.4337979,-173..."
3,147682731,"POLYGON ((-179.416732200000013 68.8980865,-179..."
4,285480537,"POLYGON ((-169.825218400000011 66.1612463,-169..."


### Coordinate transformation

In [6]:
df['polygon'] = df.apply(lambda x: wkt.loads(str(x['polygon'])), axis = 1) 

In [7]:
df.head()

,OSM_ID,polygon
0,281927594,"POLYGON ((-179.400757 68.8866838, -179.4009909..."
1,309129277,"POLYGON ((-175.8397009 65.0106686, -175.839590..."
2,129101699,"POLYGON ((-173.2124505 64.4337979, -173.212945..."
3,147682731,"POLYGON ((-179.4167322 68.89808650000001, -179..."
4,285480537,"POLYGON ((-169.8252184 66.1612463, -169.825045..."


### Coordinate system for GeoPandas

In [8]:
crs = '+ proj = longlat + datum = WGS84' #SOmetimes it`s necessary

In [8]:
gdf = gpd.GeoDataFrame(df['polygon'], geometry='polygon') 

In [9]:
def style_function(feature):
    return {
        'fillColor': '#ffaf00',
        'color': 'blue',
        'weight': 1.5,
        'dashArray': '5, 5'
    }


def highlight_function(feature):
    return {
        'fillColor': '#ffaf00',
        'color': 'green',
        'weight': 3,
        'dashArray': '5, 5',
    }

### Map creation

In [10]:
m = folium.Map(location=[68.8866838, -179.400757], zoom_start=14)
plg = folium.GeoJson(gdf.to_json(), name='polygons', style_function=style_function, highlight_function=highlight_function )
plg.add_child(folium.Popup('something on click'))
plg.add_to(m)
folium.LayerControl().add_to(m)


### Final result

In [11]:
m